In [1]:
import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from browser import getBrowserDriver

browser, waiter = getBrowserDriver()

# Pagina Inicial
el_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/a')))
el_extrajudicial.click()

el_serventias_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/ul/li[1]/a')))
el_serventias_extrajudicial.click()

In [2]:
def getEstados():
    el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
    el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

    return el_mapa_ufs_geral, el_mapa_ufs

def getMunicipios():
    el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
    el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

    return el_mapa_muns_geral, el_mapa_muns

In [3]:
# # Pagina Mapa

# # Selecionando elementos
# el_mapa_ufs_geral, el_mapa_ufs = getEstados()

# # Selecionando estados
# for uf in range(1, len(el_mapa_ufs)):
#     el_mapa_ufs[uf].click()
#     time.sleep(2)

#     # Selecionando municipios
#     el_mapa_muns_geral, el_mapa_muns = getMunicipios()

#     for num in range(1, len(el_mapa_muns)):
#         el_mapa_muns[num].click()

#         el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
#         el_pesquisar.click()  
#         time.sleep(1)      

#         browser.back()
#         time.sleep(1)  

#         # Resetando para conseguir clicar novamente
#         el_mapa_ufs_geral, el_mapa_ufs = getEstados()
#         el_mapa_ufs[uf].click()
#         el_mapa_muns_geral, el_mapa_muns = getMunicipios()

#     time.sleep(1)

In [4]:
# Pagina Mapa

# Selecionando estados
el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

el_mapa_ufs[-10].click()

# Selecionando municipios
el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

el_mapa_muns[19].click()

# Pesquisar
el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
el_pesquisar.click()

In [5]:
# # Pagina Cartorios 

# # Selecionando dados
# el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
# el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

# for info in range(0, len(el_infos)):
#     if((info + 1) % 2 == 0):
#         el_infos[info].click()
#         el_infos[info].click()
#     else:
#         el_infos[info].click()

#     time.sleep(2)
#     browser.back()
#     time.sleep(2)
    
#     el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
#     el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

In [6]:
# Pagina Cartorios 

# Selecionando dados
el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

el_infos[0].click()

In [ ]:
# Pagina Dados